In [1]:
import torch
import random

import torch.nn.functional as F
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F
from botorch.acquisition import ExpectedImprovement



m:\workspace\bo\botorch_contrast\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
BATCHSIZE = 20
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  #r = torch.clamp(r,0,1)
  return 1-torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  x = getSample()
  X = torch.cat((X, x), dim=0)
  y = dif(x, TARGET)
  Y = torch.cat((Y, y), dim=0)
  
  
Y =  F.normalize(Y, dim=None, p=2)

print(Y)


list(zip(X, Y))


tensor([ 0.2200,  0.1771,  0.3986,  0.3660,  0.0657, -0.1613,  0.2720,  0.3644,
         0.1252,  0.1784,  0.0493,  0.0023,  0.3703,  0.0445,  0.3266,  0.2383,
        -0.0326,  0.1846, -0.0032, -0.0510], dtype=torch.float64)


[(tensor(0.4704, dtype=torch.float64), tensor(0.2200, dtype=torch.float64)),
 (tensor(0.6143, dtype=torch.float64), tensor(0.1771, dtype=torch.float64)),
 (tensor(0.4996, dtype=torch.float64), tensor(0.3986, dtype=torch.float64)),
 (tensor(0.6738, dtype=torch.float64), tensor(0.3660, dtype=torch.float64)),
 (tensor(0.1786, dtype=torch.float64), tensor(0.0657, dtype=torch.float64)),
 (tensor(0.1864, dtype=torch.float64), tensor(-0.1613, dtype=torch.float64)),
 (tensor(0.1963, dtype=torch.float64), tensor(0.2720, dtype=torch.float64)),
 (tensor(0.2987, dtype=torch.float64), tensor(0.3644, dtype=torch.float64)),
 (tensor(0.9252, dtype=torch.float64), tensor(0.1252, dtype=torch.float64)),
 (tensor(0.4789, dtype=torch.float64), tensor(0.1784, dtype=torch.float64)),
 (tensor(0.9670, dtype=torch.float64), tensor(0.0493, dtype=torch.float64)),
 (tensor(0.9068, dtype=torch.float64), tensor(0.0023, dtype=torch.float64)),
 (tensor(0.2927, dtype=torch.float64), tensor(0.3703, dtype=torch.float64))

In [11]:
train_Y = standardize(Y)
#train_Y = Y
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [7]:

# Define the Expected Improvement (EI) acquisition function
ei = ExpectedImprovement(gp, best_f=torch.max(Y))

# 'best_observed_value' is the best observed value from your optimization process so far
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

# Optimize the EI acquisition function to find the next candidate point
candidate, acq_value = optimize_acqf(
    ei, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)

print(candidate)

tensor([[0.3920, 0.1869]])
